# Market-basket analysis of Letterboxd dataset

In [ ]:
# setup Spark
! apt-get install openjdk-8-jdk-headless -qq > /dev/null
! wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
! tar xf spark-3.1.1-bin-hadoop3.2.tgz
! pip install -q findspark
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
findspark.init("spark-3.1.1-bin-hadoop3.2")# SPARK_HOME

# get dataset
! kaggle datasets download -f actors.csv gsimonx37/letterboxd
! unzip actors.csv.zip
! rm actors.csv.zip

# get whole repo if running in google colab
! git clone https://github.com/mattia01017/movie-actor-mb-analysis
! cd movie-actor-mb-analysis
! pip install -r requirements.txt

In [ ]:
from dotenv import load_dotenv
from pyspark.sql import SparkSession

load_dotenv()
spark = SparkSession.builder\
    .appName("movie-actor-mb-analysis")\
    .getOrCreate() 

The first step is to import the dataset from the CSV file. To get a convenient representation of data for the market-basket analysis, data will be stored as key-value pairs, where the keys are the film identifiers and the values are the lists of actors that worked in the film.

In [ ]:
df = spark.read.csv("actors.csv", header=True, sep=",", mode="DROPMALFORMED")
baskets = df.rdd\
    .map(lambda x: (x["id"], x["name"]))\
    .groupByKey()